On instance, create a temporary workspace directory, fraud-detection example,

In [1]:
!mkdir workspace
!cp -r examples/fraud-detection workspace/
!cp -r examples/utils/gen-cert workspace/fraud-detection/

mkdir: cannot create directory ‘workspace’: File exists


This example has a separate private data-and-scratch directories for each user or ML node. Create the respective directories and copy data-and-scratch directory. Running this example creates a scratch directory for each user and saves the trained Swarm model in the directory at the end of the training.

In [2]:
!mkdir workspace/fraud-detection/user1 workspace/fraud-detection/user2
!mkdir workspace/fraud-detection/user3 workspace/fraud-detection/user4
!cp -r workspace/fraud-detection/data-and-scratch workspace/fraud-detection/user1/
!cp -r workspace/fraud-detection/data-and-scratch workspace/fraud-detection/user2/
!cp -r workspace/fraud-detection/data-and-scratch workspace/fraud-detection/user3/
!mv workspace/fraud-detection/data-and-scratch workspace/fraud-detection/user4/

mkdir: cannot create directory ‘workspace/fraud-detection/user1’: File exists
mkdir: cannot create directory ‘workspace/fraud-detection/user2’: File exists
mkdir: cannot create directory ‘workspace/fraud-detection/user3’: File exists
mkdir: cannot create directory ‘workspace/fraud-detection/user4’: File exists
mv: cannot move 'workspace/fraud-detection/data-and-scratch' to 'workspace/fraud-detection/user4/data-and-scratch': Directory not empty


Run the gen-cert utility to generate certificates for each Swarm component using the command, gen-cert -e <EXAMPLE-NAME> -i <HOST-INDEX>.

In [3]:
!./workspace/fraud-detection/gen-cert -e fraud-detection -i 1

Generating a RSA private key
........................................+++++
................................................................................................+++++
writing new private key to 'workspace/fraud-detection/cert/ca/ca-1-key.pem'
-----
./workspace/fraud-detection/gen-cert: line 55: local: can only be used in a function
Signature ok
subject=OU = Swarm-Learning, CN = CA-1
Getting Private key
./workspace/fraud-detection/gen-cert: line 69: local: can only be used in a function
Generating a RSA private key
..................................+++++
..........+++++
writing new private key to 'workspace/fraud-detection/cert/sn-1-key.pem'
-----
Signature ok
subject=OU = Swarm-Learning, CN = SN-1
Getting CA Private Key
Generating a RSA private key
...............................................................................................+++++
.............................+++++
writing new private key to 'workspace/fraud-detection/cert/sl-1-key.pem'
-----
Signature ok
sub

Search and replace all occurrences of <CURRENT-PATH> tag in swarm_fd_task.yaml and swop1_profile.yaml files with $(pwd).

In [4]:
!sed -i "s+<CURRENT-PATH>+$(pwd)+g" workspace/fraud-detection/swop/swop*_profile.yaml workspace/fraud-detection/swci/taskdefs/swarm_fd_task.yaml

Create a docker volume and copy Swarm Learning wheel file.

In [5]:
!docker volume rm sl-cli-lib
!docker volume create sl-cli-lib
!docker container create --name helper -v sl-cli-lib:/data hello-world
!docker cp -L lib/swarmlearning-client-py3-none-manylinux_2_24_x86_64.whl helper:/data
!docker rm helper

sl-cli-lib
sl-cli-lib
3f5e17cb846ae76badd7db8a9dd6d36911086bb816c7887f8b9961aa01c674e0
Preparing to copy...
Copying to container - 31.74kB
Successfully copied 31.74kB to helper:/data
helper


Create a docker network for SN, SWOP, SWCI, SL, and user containers running on the same host.

In [6]:
!docker network create host-1-net

e4fcde0d94385e0d9dd883add6cc5c994a46d62f324d84fde8ad56abbd8123ec


Run Swarm Network node (SN1) - sentinel node.

In [7]:
!./scripts/bin/run-sn -d --rm --name=sn1 \
--network=host-1-net --host-ip=sn1 --sentinel \
--key=workspace/fraud-detection/cert/sn-1-key.pem \
--cert=workspace/fraud-detection/cert/sn-1-cert.pem \
--capath=workspace/fraud-detection/cert/ca/capath \
--apls-ip=35.193.101.82

Preparing to copy...
Copying to container - 0B
Copying to container - 0B
Copying to container - 512B
Copying to container - 714B
Copying to container - 1.024kB
Copying to container - 1.536kB
Copying to container - 2.048kB
Successfully copied 2.048kB to 83aa8e1b64631e06555915bd15e28ff215923929008bab16272620ffdbf652fc:/tmp/container_info_file
83aa8e1b64631e06555915bd15e28ff215923929008bab16272620ffdbf652fc


In [8]:
!docker ps

CONTAINER ID   IMAGE                                                              COMMAND                  CREATED          STATUS          PORTS     NAMES
83aa8e1b6463   hub.myenterpriselicense.hpe.com/hpe_eval/swarm-learning/sn:1.2.0   "/usr/bin/python3 -c…"   23 seconds ago   Up 21 seconds             sn1


In [9]:
!docker logs -f sn1

######################################################################
##                    HPE SWARM LEARNING SN NODE                    ##
######################################################################
## © Copyright 2019-2022 Hewlett Packard Enterprise Development LP  ##
######################################################################
2023-03-02 10:50:47,039 : swarm.blCnt : INFO : Setting up blockchain layer for the swarm node: START
2023-03-02 10:50:48,688 : swarm.blCnt : INFO : Creating Autopass License Provider
2023-03-02 10:50:49,910 : swarm.blCnt : INFO : Creating license server
2023-03-02 10:50:49,911 : swarm.blCnt : INFO : Setting license servers
2023-03-02 10:50:50,084 : swarm.blCnt : INFO : Acquiring floating license 1100000380:1
2023-03-02 10:51:09,366 : swarm.SN : INFO : SMLETHNode: Starting GETH ... 
2023-03-02 10:51:19,397 : swarm.SN : WARNING : SMLETHNode: Enode list is empty: Node is standalone
2023-03-02 10:55:02,767 : swarm.SN : INFO : SMLETHNode: Sta

Use the Docker logs command to monitor the Sentinel SN node and wait for the node to finish initializing.
The Sentinel node is ready when the following messages appear in the log output:

swarm.blCnt : INFO : Starting SWARM-API-SERVER on port: 30304


according to environment, modify IP and proxy in the profile files under workspace/fraud-detection/swop folder.

Run Swarm Operator node (SWOP1).

In [10]:
!./scripts/bin/run-swop -d --rm --name=swop1 \
--network=host-1-net --usr-dir=workspace/fraud-detection/swop \
--profile-file-name=swop1_profile.yaml \
--key=workspace/fraud-detection/cert/swop-1-key.pem \
--cert=workspace/fraud-detection/cert/swop-1-cert.pem \
--capath=workspace/fraud-detection/cert/ca/capath \
-e SWOP_KEEP_CONTAINERS=True \
-e http_proxy= -e https_proxy= --apls-ip=35.193.101.82

Preparing to copy...
Copying to container - 0B
Copying to container - 0B
Copying to container - 512B
Copying to container - 716B
Copying to container - 1.024kB
Copying to container - 1.536kB
Copying to container - 2.048kB
Successfully copied 2.048kB to b1c62e329706f594cfbe5b518afdf57d2e0689df22a6d25640b8cd0eb400ee02:/tmp/container_info_file
b1c62e329706f594cfbe5b518afdf57d2e0689df22a6d25640b8cd0eb400ee02


Run SWCI node (SWCI1). It creates, finalizes and assigns below task to task-framework for sequential execution:
user_env_tf_build_task: Builds TensorFlow based Docker image for ML node to run model training.

swarm_fd_task: Create containers out of ML image, and mounts model and data path to run Swarm training.

In [11]:
!./scripts/bin/run-swci -ti --rm --name=swci1 \
--network=host-1-net --usr-dir=workspace/fraud-detection/swci \
--init-script-name=swci-init \
--key=workspace/fraud-detection/cert/swci-1-key.pem \
--cert=workspace/fraud-detection/cert/swci-1-cert.pem \
--capath=workspace/fraud-detection/cert/ca/capath \
-e http_proxy= -e https_proxy= --apls-ip=35.193.101.82

Preparing to copy...
Copying to container - 0B
Copying to container - 0B
Copying to container - 512B
Copying to container - 716B
Copying to container - 1.024kB
Copying to container - 1.536kB
Copying to container - 2.048kB
Successfully copied 2.048kB to 27ce6cd5c9f2fce3b78113c9fa37f948ad3ea2e7b7bdc47e91e905affb8ac5e8:/tmp/container_info_file
27ce6cd5c9f2fce3b78113c9fa37f948ad3ea2e7b7bdc47e91e905affb8ac5e8
######################################################################
##                   HPE SWARM LEARNING SWCI NODE                   ##
######################################################################
## © Copyright 2021, 2022 Hewlett Packard Enterprise Development LP ##
######################################################################
SWCI:0 > ######################################################################
SWCI:0 > # (C)Copyright 2021,2022 Hewlett Packard Enterprise Development LP
SWCI:0 > ######################################################################
S

Four nodes of Swarm training are automatically started when the run task (swarm_fd_task) gets assigned and executed. <br>Open a new terminal on host-1 and monitor the Docker logs of ML nodes for Swarm training. Swarm training ends with the following log message:
**SwarmCallback : INFO : All peers and Swarm training rounds finished. Final Swarm model was loaded.**<br>
Final Swarm model is saved inside each user’s private scratch directory, which is workspace/fraud-detection/user<id>/data-and-scratch/scratch on both the hosts. <br>All the dynamically spawned SL and ML nodes exits after Swarm training. The SN and SWOP nodes continues to run.

Go to "Home 
Page" of Jupyter notebook
click on New
click on terminal - it will open new terminal window in new browsing tab
execute following command

docker ps

it will show you sn1,swop1,swci1, 4 sl and 4 ml container

docker logs -f demo-swarm_fd_task-u-0-034698c2bbf002b7

might be your container name is different so instead demo-swarm_fd_task-u-0-034698c2bbf002b7 use your container name.

In [12]:
!docker ps -a

CONTAINER ID   IMAGE                                                                COMMAND                  CREATED          STATUS                      PORTS     NAMES
f9f5e8f52860   user-env-tf2.7.0-swop                                                "python3 model/fraud…"   6 minutes ago    Exited (0) 3 minutes ago              demo-swarm_fd_task-u-3-47a70240030fb34a
4dc7b0fe32d9   hub.myenterpriselicense.hpe.com/hpe_eval/swarm-learning/sl:1.2.0     "/usr/bin/python3 -c…"   6 minutes ago    Exited (0) 3 minutes ago              demo-swarm_fd_task-s-3-47a70240030fb34a
6b5f7eb8576e   user-env-tf2.7.0-swop                                                "python3 model/fraud…"   6 minutes ago    Exited (0) 3 minutes ago              demo-swarm_fd_task-u-2-97d1c19f2bb06f18
3bd7ccacbfa8   hub.myenterpriselicense.hpe.com/hpe_eval/swarm-learning/sl:1.2.0     "/usr/bin/python3 -c…"   6 minutes ago    Exited (0) 3 minutes ago              demo-swarm_fd_task-s-2-97d1c19f2bb06f18
50933d74181f  

In [13]:
!docker logs demo-swarm_fd_task-u-2-97d1c19f2bb06f18

2023-03-02 10:59:26.745421: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
***** Starting model = fraud-detection
----------------------------------------------------------------
loading train dataset data-and-scratch/app-data/SB19_CCFDUBL_BAL_TRAIN_2C.csv ..
size of training Data set : 684
----------------------------------------------------------------
loading test dataset data-and-scratch/app-data/SB19_CCFDUBL_BAL_TEST_2C.csv ..
----------------------------------------------------------------
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 31        
              

In [14]:
!docker logs demo-swarm_fd_task-s-3-47a70240030fb34a

######################################################################
##                    HPE SWARM LEARNING SL NODE                    ##
######################################################################
## © Copyright 2019-2022 Hewlett Packard Enterprise Development LP  ##
######################################################################
2023-03-02 10:59:29,226 : swarm.mlApp : INFO : Creating Autopass License Provider
2023-03-02 10:59:33,546 : swarm.mlApp : INFO : Creating license server
2023-03-02 10:59:33,546 : swarm.mlApp : INFO : Setting license servers
2023-03-02 10:59:33,579 : swarm.mlApp : INFO : Acquiring floating license 1100000378:1
2023-03-02 10:59:35,393 : swarm.mlApp : INFO : Opening pipes to communicate with user container ...
2023-03-02T10:59:35.394630 /tmp/hpe-swarm/demo.3.request.pipe: File exists
2023-03-02T10:59:35.395121 /tmp/hpe-swarm/demo.3.response.pipe: File exists
2023-03-02 10:59:35,396 : swarm.mlCnt : INFO : Setting up SL Container :  START
202

clean up

In [15]:
!./scripts/bin/stop-swarm

Stopping all running Swarm Learning containers
No running Swarm Learning containers found
Removing all Swarm Learning containers
4dc7b0fe32d9
3bd7ccacbfa8
6b569090a108
22995f473a32
Stopping all running Swarm Network containers
83aa8e1b6463
Removing all Swarm Network containers
Error response from daemon: removal of container 83aa8e1b6463 is already in progress
Stopping all running Swarm Command Interface containers
No running Swarm Command Interface containers found
Removing all Swarm Command Interface containers
No Swarm Command Interface containers found
Stopping all running Swarm Operator containers
b1c62e329706
Removing all Swarm Operator containers
No Swarm Operator containers found


In [16]:
!docker network rm host-1-net

host-1-net


In [17]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
